In [ ]:
%pip install tensorflow-macos tensorflow-metal # macOS용 tf 설치

In [10]:
import tensorflow as tf

# 데이터 불러오기
mnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 0~255 범위의 픽셀값을 0~1 범위로 노멀라이즈
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
from tensorflow.keras.layers import Dense # type: ignore
from tensorflow.keras.optimizers import Adam # type: ignore

# 신경망 모델 정의하기
class Model(tf.keras.Model):
    def __init__(self):
        super(Model, self).__init__()
        self.input_layer = Dense(256, activation='relu', input_shape=(784,))
        self.hidden_layer = Dense(128, activation='relu')
        self.output_layer = Dense(10, activation='softmax')
        
    def call(self, x):
        x = self.input_layer(x)
        x = self.hidden_layer(x)
        x = self.output_layer(x)
        return x

model = Model()

# 오류함수, 옵티마이저 정의하기
cross_entropy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
optimizer = Adam(1e-4)


In [9]:
# 미니배치 사이즈 설정
batch_size = 32
num_train_data = x_train.shape[0]
num_test_data = x_test.shape[0]

num_epochs = 10
for e in range(num_epochs):
    # 1 에포크 동안 훈련 진행
    for i in range(num_train_data // batch_size):
        # 미니배치 하나만큼 데이터 불러오기
        x_batch = x_train[i*batch_size:(i+1)*batch_size]
        y_batch = y_train[i*batch_size:(i+1)*batch_size]
        
        # (batch_size, 28, 28) -> (batch_size, 28*28)로 shape 변경
        x_batch = x_batch.reshape(-1, 28*28)
        
        # 계산 과정을 기록하기 위해 tape scope 선언
        model_params = model.trainable_variables  # Model -> model로 변경 (클래스가 아닌 인스턴스 사용)
        with tf.GradientTape() as tape:
            # 모델을 통한 예측
            predicts = model(x_batch)  # Model -> model로 변경
            # 오류 계산
            losses = cross_entropy(y_batch, predicts)
        # tape를 통한 그래디언트 계산
        grads = tape.gradient(losses, model_params)
        # 그래디언트를 통해 모델 업데이트
        optimizer.apply_gradients(zip(grads, model_params))
    
    # 에폭 종료 시 진행 상황 출력
    print(f"에폭 {e+1}/{num_epochs} 완료")
    
    # 에폭마다 테스트 정확도 확인 (선택 사항)
    if (e+1) % 1 == 0:
        # 테스트 데이터로 정확도 평가
        test_batch = x_test.reshape(-1, 28*28)
        test_preds = model(test_batch)
        test_preds_classes = tf.argmax(test_preds, axis=1)
        test_accuracy = tf.reduce_mean(tf.cast(tf.equal(test_preds_classes, y_test), tf.float32))
        print(f"테스트 정확도: {test_accuracy.numpy():.4f}")

에폭 1/10 완료
테스트 정확도: 0.8364
에폭 2/10 완료
테스트 정확도: 0.8522
에폭 3/10 완료
테스트 정확도: 0.8601
에폭 4/10 완료
테스트 정확도: 0.8654
에폭 5/10 완료
테스트 정확도: 0.8707
에폭 6/10 완료
테스트 정확도: 0.8743
에폭 7/10 완료
테스트 정확도: 0.8769
에폭 8/10 완료
테스트 정확도: 0.8771
에폭 9/10 완료
테스트 정확도: 0.8781
에폭 10/10 완료
테스트 정확도: 0.8811
